In [1]:
import consMDP
import json
from decimal import Decimal

In [2]:
file_path = 'NYCstreetnetwork.json'
with open(file_path,'r') as f:
        graph_file = json.load(f)

In [3]:
def convert(g):
    states = []
    actions = dict()

    mdp = consMDP.ConsMDP()

    for node in g["nodes"]:
        if node["action"]:
            actions[node["label"]] = dict()
        else:
            states.append(node)

    for s in states:
        mdp.new_state(s["reload"], s["label"])

    for edge in g["edges"]:
        fr = edge["tail"]
        to = edge["head"]
        if to in actions:
            actions[to]["from"] = fr
            actions[to]["cons"] = edge["consumption"]
        else:
            dist = actions[fr].get("dist")
            to = mdp.state_with_label(to)
            if dist is None:
                actions[fr]["dist"] = dict()
            actions[fr]["dist"][to] = Decimal(f'{edge["probability"]}')

    for label, a in actions.items():
        fr = mdp.state_with_label(a["from"])
        mdp.add_action(fr, a["dist"], label, a["cons"])

    return mdp

In [4]:
def get_target_set(g, mdp):
    T = set()
    for t in g["T"]:
        T.add(mdp.state_with_label(t["label"]))
    return T

In [5]:
%%timeit
m = convert(graph_file)
targets = get_target_set(graph_file, m)

41.8 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
m = convert(graph_file)
targets = get_target_set(graph_file, m)
type(m)

consMDP.ConsMDP

In [7]:
%%timeit
m.get_minInitCons(recompute=True);

342 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
m.get_minInitCons(30, recompute=True);

181 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
m.get_safeReloads(100, recompute=True)

375 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
m.get_positiveReachability(targets, 100, recompute=True)

1.05 s ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
